# Download some IR for Jazz and me over Congo basin

Lats -10-10, Lons 10-30


In [83]:
import xarray as xr 
import matplotlib.pyplot as plt 

In [62]:
# a MONTH of hourly URLs, format .../GPM_MERGIR.1/2012/200/merg_2012071800_4km-pixel.nc4

from datetime import datetime, timedelta

def generate_mergir_urls(year=2012, month=8, day_start=1, day_end=2):
    # Start date
    start_date = datetime(year, month, day_start)
    urls = []
    filenames=[]
    
    current_date = start_date
    
    while current_date.month == month:
        # Format: YYYYDDD (day of year)
        doy = current_date.strftime('%Y%j')  # %j gives day of year with zero-padding
        mm = current_date.strftime('%m')  # %j gives day of year with zero-padding
        dd = current_date.strftime('%d')  # %j gives day of year with zero-padding
        hh = current_date.strftime('%H')  # %j gives day of year with zero-padding
        
        filename = f"merg_{year}{mm}{dd}{hh}_4km-pixel.nc4"
        url = f"https://disc2.gesdisc.eosdis.nasa.gov/opendap/MERGED_IR/GPM_MERGIR.1/{year}/{int(doy[4:]):03d}/{filename}"
        urls.append(url)
        filenames.append(filename)
        
        # Next hour
        current_date += timedelta(hours=1)
        if current_date.day > day_end:
            break

    return urls,filenames

In [85]:
# Generate list for August 1 to August 31, 2012
myurls,filenames = generate_mergir_urls(2025, 9, 1, 28)
myurls,filenames = generate_mergir_urls(2024, 9, 1, 30)

In [86]:
for i,url in enumerate(myurls): 
    ds = xr.open_dataset(url)
    congo = ds.sel(lat=slice(-10,10), lon=slice(10,30))
# Hourly 8km (every other pixel and time 
# like /Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2025091701_4km-pixel.nc4
    congo.Tb[::2,::2,::2].to_netcdf('/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/'+filenames[i]) 
    print('/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/'+filenames[i])

/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090100_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090101_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090102_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090103_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090104_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090105_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090106_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090107_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090108_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090109_4km-pixel.nc4
/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/merg_2024090110_4km-pixel.nc4

In [87]:
dsm = xr.open_mfdataset('/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/*.nc4')
dsm.to_netcdf('/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/2024-09_hourly_congo_Tb.nc') 

In [88]:
ds = xr.open_dataset('/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/2025-09_hourly_congo_Tb.nc') 

(ds.Tb <238).mean(dim='time').plot(); 
plt.title('Mean coverage by Tb <238K, Sep 2025');

FileNotFoundError: [Errno 2] No such file or directory: '/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/2025-09_hourly_congo_Tb.nc'

In [ ]:
ds = xr.open_dataset('/Users/bmapes/Box/Jazz-Brian-Africa-satellite/MERGED_IR_DATA/2024-09_hourly_congo_Tb.nc') 

(ds.Tb <238).mean(dim='time').plot(); 
plt.title('Mean coverage by Tb <238K, Sep 2024');

In [4]:
# Gemini made this "sketch" of a notebook for Meteosat downloads

# Install the necessary libraries
!pip install --user 'satpy[all]'
!pip install --user aiohttp

# Import necessary libraries
import satpy
from satpy import find_files_and_readers
from satpy.utils import get_geos_area_def

# --- User-configurable variables ---
start_time = '20230801'
end_time = '20230901'
satellite_name = 'Meteosat-11'

# --- Find and load data ---
# This part requires programmatic access which can be set up after account approval.
# You would get a list of files from the Data Store API for your date range.
# For now, let's assume you have a list of local files.
# raw_files = find_files_and_readers(start_time=start_time, end_time=end_time,
#                                    readers='seviri_l1b_native', base_dir='/path/to/data/')

# Let's assume you have downloaded a single file for demonstration
file_path = "MSG4-SEVI-MSG15-0100-NA-202308010000.1.0.tar" 
scn = satpy.Scene(reader='seviri_l1b_native', filenames=file_path)

# --- Define Congo Basin bounding box and resample ---
# This is where you create your lat-lon box
area_id = 'my_congo_basin'
proj_params = {'proj': 'longlat', 'ellps': 'WGS84'}
area_extent = (10, -10, 35, 5) # (lon_min, lat_min, lon_max, lat_max)

area_def = get_geos_area_def(
    area_id, proj_params, area_extent, 
    width=1000, height=1000
)

# Load the infrared channel
scn.load(['IR_108'])
scn_resampled = scn.resample(area_def)

# Plotting
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.imshow(scn_resampled['IR_108'], origin='lower')
plt.show()

# --- For a full month of data, you would loop through files and resample ---
# The logic would be similar to the previous NOAA GOES example.
# This approach ensures you only work with the resampled subset in memory.

  Using cached donfig-0.8.1.post1-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.7 MB/s eta 0:00:00
Using cached donfig-0.8.1.post1-py3-none-any.whl (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━ 5/8 [pyresample]  WARNING: The scripts fetch_tles and fetch_tles.py are installed in '/Users/bmapes/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━ 7/8 [satpy]  WARNING: The script satpy_retrieve_all_aux_data is installed in '/Users/bmapes/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [satpy]


ModuleNotFoundError: No module named 'satpy'